In [1]:
# Imports
import os 
import pandas as pd
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from datetime import date
from edc_appointment.models import Appointment

# data_path = '/Users/imosweu/flourish_twins/'
data_path = '/home/flourish/'

In [2]:
sc = SubjectConsent.objects.get(subject_identifier='B142-040990594-9')

In [3]:
cccs = sc.caregiverchildconsent_set.all()

In [4]:
cccs.delete()

(2, {'flourish_caregiver.CaregiverChildConsent': 2})

In [6]:
# Subject Consent
child_consents = sc.caregiverchildconsent_set.all()
child_consents_data = []

for consent in child_consents:
    data = consent.__dict__
    print(data)
    child_consents_data.append(data)
df_child_consent = pd.DataFrame(child_consents_data)

child_consents_file = 'child_consents.csv'
fpath = data_path + child_consents_file
df_child_consent.to_csv(fpath, sep='|', encoding='utf-8')

In [15]:
child_consents.delete()

(2, {'flourish_caregiver.CaregiverChildConsent': 2})

In [6]:
scs = ChildDummySubjectConsent.objects.filter(subject_identifier__startswith='B142-040990594-9').exclude(subject_identifier__endswith='5')

In [7]:
scs.delete()

(2, {'flourish_child.ChildDummySubjectConsent': 2})

In [8]:
rs = RegisteredSubject.objects.filter(subject_identifier__startswith='B142-040990594-9-').exclude(subject_identifier__endswith='5')

In [9]:
rs

<QuerySet [<RegisteredSubject: B142-040990594-9-60>, <RegisteredSubject: B142-040990594-9-10>]>

In [ ]:
ChildAssent.objects.filter(subject_identifier__startswith='B142-040990594-9').delete()

In [10]:
rs.delete()

(2, {'edc_registration.RegisteredSubject': 2})

In [12]:
IdentifierModel.objects.get(identifier='B142-040990594-9-60').delete()

(1, {'edc_identifier.IdentifierModel': 1})

In [13]:
IdentifierModel.objects.get(identifier='B142-040990594-9-10').delete()

(1, {'edc_identifier.IdentifierModel': 1})

In [ ]:
# Child Consent
import uuid
from dateutil import parser
from flourish_caregiver.models import CaregiverChildConsent
from django.contrib.sites.models import Site
from django.core.exceptions import ValidationError

fname = data_path + 'child_consents.csv'
f = open(fname, 'r')
lines = f.readlines()
headers = lines[0]
headers = headers.strip()
headers = headers.split('|')
del headers[0]
lines.pop(0)

s_consent_data = []
count = 0
for line in lines:
    line = line.strip()
    line = line.split('|')
    del line[0]
    if len(line) == len(headers):
        data = dict(zip(headers, line))
    else:
        print(lines[count - 1])
        print(line)
        print(lines[count + 1])
        raise ValidationError('Line is not the right size')
    s_consent_data.append(data)
    count += 0
    

count = 0
already_created = 0
pids_dict = {}
    
for dd in s_consent_data:
    del dd['_state']
    created = parser.parse(dd.get('created'))
    consent_datetime = parser.parse(dd.get('consent_datetime'))
    child_dob = parser.parse(dd.get('child_dob'))
    child_dob = child_dob.date()
    dd.update(
        created=created,
        consent_datetime=consent_datetime,
        child_dob=child_dob,
        subject_consent_id=sc.id
    )
    
    if dd['modified']:
        modified = parser.parse(dd.get('modified'))
        dd.update(modified=modified)
    else:
        dd.update(modified=None)
    if dd['is_verified_datetime']:
        is_verified_datetime = parser.parse(dd.get('is_verified_datetime'))
        dd.update(is_verified_datetime=is_verified_datetime)
    else:
        dd.update(is_verified_datetime=None)
    if dd.get('gender') == 'M':
        dd.update(study_child_identifier = '056-4981893-0-25')
    if dd.get('gender') == 'F':
        dd.update(study_child_identifier = '056-4981893-0-35')
        
    if dd['caregiver_visit_count']:
        caregiver_visit_count = parser.parse(dd.get('caregiver_visit_count'))
        dd.update(caregiver_visit_count=caregiver_visit_count)
    else:
        dd.update(caregiver_visit_count=0)

    try:
        CaregiverChildConsent.objects.get(identity=dd.get('identity'))
    except CaregiverChildConsent.DoesNotExist:
        old_id = dd.get('subject_identifier')
        del dd['subject_identifier']
        del dd['dob']
        
#         import pdb; pdb.set_trace()
        print(dd)
        subject_consent = CaregiverChildConsent.objects.create(**dd)

        count += 1
    else:
        already_created += 1
print(f'Total created consents {count}')
print(pids_dict)